# Ungraded Lab: Feature Engineering with Accelerometer Data

This notebook demonstrates how to prepare time series data taken from an accelerometer. We will be using the [WISDM Human Activity Recognition Dataset](http://www.cis.fordham.edu/wisdm/dataset.php) for this example. This dataset can be used to predict the activity a user performs from a set of acceleration values recorded from the accelerometer data of a smartphone.

The dataset consists of accelerometer data in the x, y, and z-axis recorded for 36 user different users. A total of 6 activities like 'Walking','Jogging', 'Upstairs', 'Downstairs', 'Sitting', 'Standing' etc. were recorded. The sensors have a sampling rate of 20Hz which means there are 20 observations recorded per second.

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
import pandas as pd
import numpy as np
import urllib

pp = pprint.PrettyPrinter()

## Extract the Data

In [ ]:
# Setup paths and filenames
working_dir = './data/'
TRANSFORM_TRAIN_FILENAME = 'transform_train'
TRANSFORM_TEST_FILENAME = 'transform_test'
TRANSFORM_TEMP_DIR = 'tft_temp'
INPUT_FILE = './data/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt'

In [ ]:
# Extract the data
!tar -xvf ./data/human_activity/raw/WISDM_ar_latest.tar.gz -C ./data/human_activity/raw/

## Inspect the Dataset

### Utilities

Since this is accelerometer data, it would be good to visualize different aspects of the measurements. You can look at the frequency of activities, or plot the measurements against time. These utility functions will help in doing that.

In [ ]:
# Visulaization Utilities
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot

def visualize_value_plots_for_categorical_feature(feature, colors=['b']):
    '''Plots a bar graph for categorical features'''
    counts = feature.value_counts()
    plt.bar(counts.index, counts.values, color=colors)
    plt.show()


def visualize_plots(dataset, activity, columns):
    '''Visualizes the accelerometer data against time'''
    features = dataset[dataset['activity'] == activity][columns][:200]
    if 'z-acc' in columns:
        features['z-acc'] = features['z-acc'].replace(regex=True, to_replace=r';', value=r'')
        features['z-acc'] = features['z-acc'].astype(np.float64)
    axis = features.plot(subplots=True, figsize=(16, 12), 
                     title=activity)

    for ax in axis:
        ax.legend(loc='lower left', bbox_to_anchor=(1.0, 0.5))

### Clean the data

You will also need to clean stray characters that may misrepresent your data. For this particular dataset, there is a semicolon at the end of each row and this will cause the z-acceleration to be interpreted as a string. Let's clean that up in the cells below.

In [ ]:
# Set up paths
RAW_DATA_PATH = 'data/human_activity/raw/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt'
CLEAN_DATA_PATH = 'data/human_activity/pipeline_data'

# Create clean data path (raw data path already exists)
!mkdir {CLEAN_DATA_PATH}

In [ ]:
# Preview the dataset. See the semicolon at the end of each line.
!head data/human_activity/raw/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt

In [ ]:
# Load the dataset and set the column names
df = pd.read_csv(RAW_DATA_PATH, header=None, names=['user_id', 'activity', 'timestamp', 'x-acc','y-acc', 'z-acc'])

# Remove semicolon at the end of every row
df['z-acc'] = df['z-acc'].replace({';':''}, regex=True)

# Write the file to the clean data path
df.to_csv(f'{CLEAN_DATA_PATH}/human_activity.csv', index=False)

In [ ]:
# See the results. The semicolon should now be removed.
!head {CLEAN_DATA_PATH}/human_activity.csv

### Histogram of Activities

You can now proceed with the visualizations. You can plot the histogram of activities and make your observations. For instance, you'll notice that there is more data for walking and jogging than other activities. This might have an effect on how your model learns each activity so you should take note of it.

In [ ]:
# Plot the histogram of activities
visualize_value_plots_for_categorical_feature(df['activity'], colors=['r', 'g', 'b', 'y', 'm', 'c'])

### Histogram of Measurements per User
You can also observe the number of measurements taken per user. From the plot below, you can see that for the 36 users in the study, the number of observations per user is mostly steady except for a few.

In [ ]:
# Plot the histogram for users
visualize_value_plots_for_categorical_feature(df['user_id'])

### Acceleration per Activity

Finally, you can plot the sensor measurements against the timestamps. You can observe that acceleration is more for activities like jogging when compared to sitting which should be the expected behaviour. If this is not the case, then there might be problems with the sensor and can make the data invalid.

In [ ]:
# Plot the measurements for `Jogging`
visualize_plots(df, 'Jogging', columns=['x-acc', 'y-acc', 'z-acc'])

In [ ]:
# Plot the measurements for `Sitting`
visualize_plots(df, 'Sitting', columns=['x-acc', 'y-acc', 'z-acc'])

## Data Pipeline

You can now feed the data into the TFX pipeline. As in the previous labs, we won't go over too much on the first few stages of the pipeline since you've already done it before.

In [ ]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './data/human_activity/pipeline_data'

In [ ]:
# Initialize the InteractiveContext.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
context = InteractiveContext(pipeline_root=_pipeline_root)

### ExampleGen

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=_data_root)

# Execute the component
context.run(example_gen)

### Preview the Ingested Dataset

In [ ]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

In [ ]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'train')

# See the contents of the `train` folder
!ls {train_uri}

In [ ]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [ ]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

### StatisticsGen

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

In [ ]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

In [ ]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

In [ ]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

### ExampleValidator

In [ ]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

In [ ]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

### Transform

In [ ]:
# Set the constants module filename
_activity_constants_module_file = 'activity_constants.py'

In [ ]:
%%writefile {_activity_constants_module_file}

# Numerical features that are marked as continuous
INT_FEATURES = ['user_id', 'timestamp']

# Feature that can be grouped into buckets
FLOAT_FEATURES = ['x-acc', 'y-acc', 'z-acc']

# Feature that the model will predict
LABEL_KEY = 'activity'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

In [ ]:
# Set the transform module filename
_activity_transform_module_file = 'activity_transform.py'

In [ ]:
%%writefile {_activity_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import activity_constants
import importlib

importlib.reload(activity_constants)

# Unpack the contents of the constants module
_INT_FEATURES = activity_constants.INT_FEATURES
_FLOAT_FEATURES = activity_constants.FLOAT_FEATURES
_LABEL_KEY = activity_constants.LABEL_KEY
_transformed_name = activity_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY],vocab_filename=_LABEL_KEY)

    for key in _FLOAT_FEATURES:
        outputs[_transformed_name(key)] = tft.scale_by_min_max(inputs[key])

    return outputs

In [ ]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_activity_transform_module_file))

# Run the component
context.run(transform)

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

## Prepare Dataset Window

Now that you have the transformed examples, you now need to prepare the dataset window for this time series data. As discussed in class, you want to group a series of measurements and that will be the feature for a particular label. In this particular case, it makes sense to group consecutive measurements and use that as the indicator for an activity. For example, if you take 80 measurements and it oscillates greatly (just like in the visualizations in the earlier parts of this notebook), then the model should be able to tell that it is a 'Running' activity. Let's implement that in the following cells using the [tf.data.Dataset.window()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window) method.

In [ ]:
# Get the URI of the output graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# Wrap output graph with TFTTransformOutput
tf_transform_output = tft.TFTransformOutput(transform_graph_uri)

In [ ]:
# Parameters
HISTORY_SIZE = 80
BATCH_SIZE = 100
SHIFT = 40

# Helper functions
def parse_function(example_proto):
    '''Parse the values from tf examples'''
    feature_spec = tf_transform_output.transformed_feature_spec()
    features = tf.io.parse_single_example(example_proto, feature_spec)
    values = list(features.values())
    values = [float(_fill_in_missing(value)) for value in values]    
    features = tf.stack(values, axis=0)
    return features

def add_mode(features):
    '''Calculate mode of activity for the current history size of elements'''
    features = tf.squeeze(features)
    unique, _, count = tf.unique_with_counts(features[:,0])
    max_occurrences = tf.reduce_max(count)
    max_cond = tf.equal(count, max_occurrences)
    max_numbers = tf.squeeze(tf.gather(unique, tf.where(max_cond)))

    #Features (X) are all features except activity (x-acc, y-acc, z-acc)
    #Target(Y) is the mode of activity values of all rows in this window
    return (features[:,1:], max_numbers)

def get_dataset(path):
    '''Get the dataset and group them into windows'''
    dataset = tf.data.TFRecordDataset(path, compression_type="GZIP")
    dataset = dataset.map(parse_function)
    dataset = dataset.window(HISTORY_SIZE, shift=SHIFT, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(HISTORY_SIZE))

    dataset = dataset.map(add_mode)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat()

    return dataset

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
    Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = '' if x.dtype == tf.string else 0
    return tf.sparse.to_dense(x, default_value)

In [ ]:
# Get the URI of the transformed examples
working_dir = transform.outputs['transformed_examples'].get()[0].uri

# Get the filename of the compressed examples
train_tfrecord_files = os.listdir(working_dir + '/train')[0]

# Full path string to the training tfrecord files
path_to_train_tfrecord_files = os.path.join(working_dir, 'train', train_tfrecord_files)

# Get the window datasets by passing the full path to the get_dataset function
train_dataset = get_dataset(path_to_train_tfrecord_files)

# Preview the results for 1 record
for x, y in train_dataset.take(1):
    print("\nFeatures (x-acc, y-acc, z-acc):\n")
    print(x)
    print("\nTarget (activity):\n")
    print(y)

You should see a sample of a dataset window above. There are 80 consecutive measurements of `x-acc`, `y-acc`, and `z-acc` that correspond to a single labeled activity. Moreover, you also set it up to be in batches of 100 windows. This can now be fed to train an LSTM so it can learn how to detect activities based on 80-measurement windows.

## Wrap Up

In this lab, you were able to prepare time-series data from an accelerometer to transformed features that are grouped into windows to make predictions. The same concept can be applied to any data where it makes sense to take a few seconds of measurements before the model makes a prediction. 